# Comparing GraphSAGE to CASC

## Introduction


In [ ]:
import os
import networkx as nx
import pandas as pd
import graspologic
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


data_dir = os.path.expanduser("~/Documents/Neural Data Design/cora")

## Load the CORA Dataset